<a href="https://colab.research.google.com/github/MJSahebnasi/temporal-network-platform/blob/main/thesis_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import networkx as nx

from typing import List, Type

# **Model**

## Events

In [5]:
class Event:
  def __init__(self, t) -> None:
    self.time = t

class NodeEvent(Event):
  def __init__(self, t, node) -> None:
    super().__init__(self, t)
    self.node = node

class NodeEntranceEvent(NodeEvent):
  def __init__(self, t, node, neighbors) -> None:
    super().__init__(self, t, node)
    self.neighbors = neighbors

  # need id (or other info) for edges?

class NodeNaturalDeathEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(self, t, node)

class NodeAttackEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(self, t, node)

class NodeOnEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(self, t, node)

class NodeOffEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(self, t, node)


## Simulation

In [20]:
class Simulation:
  
  def __init__(self, G, duration) -> None:
    self.G = G
    self.duration = duration

    self.has_entrance_process = False
    self.entrance_process_duration = None
    self.entrance_time_generator = None
    # self.node_time_pairs = None

    self.has_onoff_process = False
    self.onoff_process_duration = None
    self.on_time_generator = None
    self.off_time_generator = None
    # self.node_onoff_times_arrays

    self.has_node_lifespan = False
    self.lifespan_generator = None
    # self.node_lifespan_pairs = None

    self.has_attack_process = False
    self.attack_time_generator = None
    # self.attack_time_node_pairs = None

  def create(self, G, simulation_duration):
    self.__init__(G, simulation_duration)
    return self

  def with_entrance_process(self, entrance_process_duration, entrance_time_generator):
    ### TODO: parameter validation ###
    self.has_entrance_process = True
    self.entrance_process_duration = entrance_process_duration
    self.entrance_time_generator = entrance_time_generator
    return self

  def with_node_onoff_process(self, onoff_process_duration, on_time_generator, off_time_generator):
    ### TODO: parameter validation ###
    self.has_onoff_process = True
    self.onoff_process_duration = onoff_process_duration
    self.on_time_generator = on_time_generator
    self.off_time_generator = off_time_generator
    return self

  def with_node_lifespans(self, lifespan_generator):
    ### TODO: parameter validation ###
    self.has_node_lifespan = True
    self.lifespan_generator = lifespan_generator
    return self

  def with_attack_process(self, attack_time_generator):
    ### TODO: parameter validation ###
    self.has_attack_process = True
    self.attack_time_generator = attack_time_generator
    return self
  

In [22]:
s = Simulation().create('G', 100)

TypeError: ignored